# Assignment 2 Linear Programing 

In [7]:
%!pip install xlrd
%!pip install openpyxl

#import for data processings
import pandas as pd
import numpy as np
#linear programer 
from gurobipy import *


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ------------------------------------ 250.0/250.0 kB 414.1 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Question 2 part c

In [84]:
#import data
df = pd.read_excel("data.xlsx")

In [149]:
#extract data
xs = df["x"]
ys = df["y"]
qs = df["q"]

In [150]:
xs = np.array(xs)
ys = np.array(ys)
qs = np.array(qs)
n_total = xs.shape[0]

In [87]:
#creat model
m = Model()

# The variables are of shape 100x2 and looklike [[u,v],[alpha_1, beta_1],[ alpha_2,beta_2] .... [alpha_n, beta_n]]
x = m.addMVar((n_total+1,2), name = "x")

m.setObjective(sum(qs[i]*x[i+1,0] + qs[i]*x[i+1,1] for i in range(n_total)))
# add constraints
for i in range(n_total):
    m.addConstr(x[i+1,0] + x[0,0] >= xs[i])
    m.addConstr(x[i+1,0] - x[0,0] >= -xs[i])
    m.addConstr(x[i+1,1] + x[0,1] >= ys[i])
    m.addConstr(x[i+1,1] - x[0,1] >= -ys[i])
#run model
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Athlon Gold 3150U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 396 rows, 200 columns and 792 nonzeros
Model fingerprint: 0xd8b42a98
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 1e+02]
Presolve time: 0.02s
Presolved: 396 rows, 200 columns, 792 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.382000e+04   0.000000e+00      0s
     196    5.2515000e+04   0.000000e+00   0.000000e+00      0s

Solved in 196 iterations and 0.04 seconds (0.00 work units)
Optimal objective  5.251500000e+04


In [89]:
print("Objective = {:.2f},u = {:.2f}, v = {:.2f}".format( m.objVal, x[0,0].x, x[0,1].x))

Objective = 52515.00,u = 70.00, v = 68.00


## Question 3 part a

In [209]:
#calculates new k using the centroids given my LP solver
def get_newK(xs,ys, centroids, n_total,iter):
    distances = np.zeros((n_total,5))
    #calculate manhatttan distance of each point from each of the centre
    for i in range(5):
        distances[:,i] = np.absolute(xs - centroids[iter][2*i]) + np.absolute(ys - centroids[iter][2*i+1])
    new_k = np.argmin(distances, axis = 1)
    new_k  
    return new_k 

In [225]:
#generate random allocation
k = np.array( [np.random.randint(0, 5) for i in range(n_total)])
cost_dict = {}
centroids = []

#initial large centroid diff to start loop
centroid_diff = 500
iters = 0
while centroid_diff >= 0.000001:
    iter_centorids = []
    for i in range(5):
        # solve problem
        #creat model
        m2 = Model()
        iter = "iter p{}".format(iters)
        if iter in cost_dict.keys():
            pass
        else:
            cost_dict[iter] = 0
        xs_temp = xs[k == i]
        qs_temp = qs[k == i]
        ys_temp = ys[k == i]
        n_temp = xs_temp.shape[0]

        # The variables are of shape (n+1)x2 and looklike [[u,v],[alpha_1, beta_1],[ alpha_2,beta_2] .... [alpha_n, beta_n]]
        x = m2.addMVar((n_temp+1,2), name = "x")
        m2.setObjective(sum(qs_temp[i]*(x[i+1,0] +x[i+1,1]) for i in range(n_temp)))

        # add constraints
        for i in range(n_temp):
            m2.addConstr(x[i+1,0] + x[0,0] >= xs_temp[i])
            m2.addConstr(x[i+1,0] - x[0,0] >= -xs_temp[i])
            m2.addConstr(x[i+1,1] + x[0,1] >= ys_temp[i])
            m2.addConstr(x[i+1,1] - x[0,1] >= -ys_temp[i])
        #run model
        m2.optimize()
        cost_dict[iter] += float(m2.objVal)
        iter_centorids.extend([float(x[0,0].x), float(x[0,1].x)])
    centroids.append(iter_centorids)
    centroids_arr = np.array(centroids)
    k = get_newK(xs,ys,centroids_arr, n_total, iters)
    #check movement of centroids
    if iters>=1:
        centroid_diff = np.mean(np.absolute(centroids_arr[iters,:] - centroids_arr[iters-1,:]))
    print("centoid_diff", centroid_diff)
    #keep track of iters
    iters+=1

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Athlon Gold 3150U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 56 rows, 30 columns and 112 nonzeros
Model fingerprint: 0x926cf9d6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 1e+02]
Presolve time: 0.01s
Presolved: 56 rows, 30 columns, 112 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.959000e+03   0.000000e+00      0s
      28    7.8570000e+03   0.000000e+00   0.000000e+00      0s

Solved in 28 iterations and 0.02 seconds (0.00 work units)
Optimal objective  7.857000000e+03
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Athlon Gold 3150U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores

In [222]:
# New k allocation
k = k+1
print(k)
#save to excel
df = pd.DataFrame(k)
df.to_excel(excel_writer = "finalk.xlsx")

[4 1 2 1 2 2 3 2 3 1 1 4 5 5 2 1 4 4 1 2 4 1 5 2 1 5 1 2 5 2 2 1 2 1 1 4 3
 3 3 4 4 3 5 4 4 5 4 1 1 4 5 3 1 2 5 4 4 4 1 1 2 5 4 5 5 1 5 1 3 3 2 2 5 4
 2 3 5 1 4 5 2 2 1 2 4 4 1 1 1 2 2 1 1 4 4 4 1 4 1]


In [223]:
centroids_arr[-1]

array([ 68.,  73.,  99.,  45.,  68.,  13.,  36.,  80., 106.,  75.])

In [224]:
cost_dict

{'iter p0': 49792.0,
 'iter p1': 31700.0,
 'iter p2': 28308.0,
 'iter p3': 24939.0,
 'iter p4': 24835.0,
 'iter p5': 24810.0,
 'iter p6': 24810.0}